# Working Models

We can attempt, purely for fun, to use the newly developed models to predict the diagnoses for certain baseline visits where the diagnosis is actually missing.

## Patients with a null diagnosis at baseline visit

In [1]:
import os
import joblib
import numpy as np
import pandas as pd 

from CogniPredictAD.preprocessing import ADNIPreprocessor, ADNICleaner, ADNITransformator, knn_impute_group
from sklearn.impute import KNNImputer

pd.set_option("display.max_rows", 116)
pd.set_option("display.max_columns", 116)
pd.set_option("display.max_info_columns", 116) 

# Open the evaluation_dataset with pandas
evaluation_dataset = pd.read_csv("../data/ADNIMERGE.csv")
# Filter baseline ("bl") visits with missing diagnoses
evaluation_dataset = evaluation_dataset[(evaluation_dataset['VISCODE'] == 'bl') & (evaluation_dataset['DX_bl'].isna())]

evaluation_dataset.reset_index(drop=True, inplace=True)

cleaner = ADNICleaner(evaluation_dataset)
transformer = ADNITransformator(evaluation_dataset)

display(evaluation_dataset)

C:\Users\Utente\AppData\Local\Temp\ipykernel_22192\2513586185.py:14: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  evaluation_dataset = pd.read_csv("../data/ADNIMERGE.csv")


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,FBB,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,DIGITSCOR_bl,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,IMAGEUID_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,6712,ADNI3,ADNI3,019_S_6712,19,bl,2019-04-24,NaN,90.6,Male,17,Not Hisp/Latino,White,Married,NaN,0.735309,NaN,NaN,1.5838,NaN,NaN,NaN,7.0,29.33,41.33,8.0,20.0,8.0,0.0,2.0,100.00000,0.0,NaN,300.0,16.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.375,1.66667,1.50000,2.0,2.50000,2.50,2.05263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-19.747900,-18.115400,2019-04-24,7.0,29.33,41.33,8.0,20.0,8.0,0.0,2.0,100.00000,0.0,NaN,300.0,16.0,-19.747900,-18.115400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.375,1.66667,1.50000,2.0,2.50000,2.50,2.05263,NaN,NaN,NaN,0.735309,NaN,NaN,1.5838,0.0,0.0,0,0,2023-07-07 05:00:04.0
1,6880,ADNI3,ADNI3,137_S_6880,137,bl,2020-12-09,NaN,66.3,Male,16,Not Hisp/Latino,White,Married,0.0,NaN,NaN,NaN,1.0646,NaN,NaN,NaN,0.5,3.67,7.67,4.0,28.0,46.0,6.0,4.0,33.33330,8.0,NaN,61.0,0.0,28.0,2.000,1.22222,1.00000,1.4,1.16667,1.75,1.42105,1.000,1.00000,1.00000,1.0,1.00000,1.00,1.00000,NaN,Cross-Sectional FreeSurfer (6.0),1360078.0,17827.4,7590.5,1000750.0,4368.0,16228.0,21865.0,1407420.0,NaN,-4.268840,-2.718210,2020-12-09,0.5,3.67,7.67,4.0,28.0,46.0,6.0,4.0,33.33330,8.0,NaN,61.0,0.0,-4.268840,-2.718210,NaN,Cross-Sectional FreeSurfer (6.0),1360078.0,17827.4,7590.5,1000750.0,4368.0,16228.0,21865.0,1407420.0,28.0,2.000,1.22222,1.00000,1.4,1.16667,1.75,1.42105,1.000,1.00000,1.00000,1.0,1.00000,1.00,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,1.0646,0.0,0.0,0,0,2023-07-09 05:25:37.0
2,6883,ADNI3,ADNI3,137_S_6883,137,bl,2020-12-21,NaN,64.5,Female,18,Not Hisp/Latino,White,Divorced,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,3.33,5.33,2.0,29.0,53.0,2.0,3.0,25.00000,11.0,NaN,68.0,0.0,NaN,1.375,2.11111,1.33333,1.6,1.40000,1.25,1.56757,1.250,1.11111,1.00000,1.0,1.16667,1.00,1.10526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.420490,-0.088403,2020-12-21,0.5,3.33,5.33,2.0,29.0,53.0,2.0,3.0,25.00000,11.0,NaN,68.0,0.0,-0.420490,-0.088403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.375,2.11111,1.33333,1.6,1.40000,1.25,1.56757,1.250,1.11111,1.00000,1.0,1.16667,1.00,1.10526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,2023-07-07 05:00:05.0
3,6912,ADNI3,ADNI3,067_S_6912,67,bl,2021-07-26,NaN,60.3,Female,16,Not Hisp/Latino,More than one,Divorced,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.33,15.33,7.0,28.0,35.0,3.0,7.0,87.50000,8.0,NaN,119.0,NaN,24.0,3.250,2.22222,1.16667,2.0,3.33333,3.50,2.55263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cross-Sectional FreeSurfer (6.0),1433667.0,29464.5,7214.5,1153330.0,3661.0,20394.0,21953.0,1645580.0,NaN,-6.496450,-5.972080,2021-07-26,2.0,8.33,15.33,7.0,28.0,35.0,3.0,7.0,87.50000,8.0,NaN,119.0,NaN,-6.496450,-5.972080,NaN,Cross-Sectional FreeSurfer (6.0),1433667.0,29464.5,7214.5,1153330.0,3661.0,20394.0,21953.0,1645580.0,24.0,3.250,2.22222,1.16667,2.0,3.33333,3.50,2.55263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [2]:
evaluation_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 116 columns):
 #    Column                    Non-Null Count  Dtype  
---   ------                    --------------  -----  
 0    RID                       11 non-null     int64  
 1    COLPROT                   11 non-null     object 
 2    ORIGPROT                  11 non-null     object 
 3    PTID                      11 non-null     object 
 4    SITE                      11 non-null     int64  
 5    VISCODE                   11 non-null     object 
 6    EXAMDATE                  11 non-null     object 
 7    DX_bl                     0 non-null      object 
 8    AGE                       11 non-null     float64
 9    PTGENDER                  11 non-null     object 
 10   PTEDUCAT                  11 non-null     int64  
 11   PTETHCAT                  11 non-null     object 
 12   PTRACCAT                  11 non-null     object 
 13   PTMARRY                   11 non-null     object 


In [3]:
pretrain = pd.read_csv(("../data/pretrain.csv"))
print(pretrain.columns)

Index(['DX', 'AGE', 'PTGENDER', 'PTEDUCAT', 'APOE4', 'CDRSB', 'ADAS11',
       'ADAS13', 'ADASQ4', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning',
       'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'LDELTOTAL', 'TRABSCOR',
       'FAQ', 'mPACCdigit', 'mPACCtrailsB', 'Ventricles', 'Hippocampus',
       'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'MOCA',
       'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan',
       'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'ABETA', 'TAU', 'PTAU', 'FDG', 'PTDEMOGROUP', 'MARRIED'],
      dtype='object')


## Same Preprocessing

For our *"evaluation_dataset"*, we need to have the same columns and impute NULL values. In practice, we need to subject it to some of the preprocessing already done for the training data.

In [4]:
evaluation_dataset = evaluation_dataset[pretrain.columns.intersection(evaluation_dataset.columns)]

In [5]:
pretrain.drop(columns=['DX'], axis=1, inplace=True)
y = evaluation_dataset['DX']
evaluation_dataset.drop(columns=['DX'], axis=1, inplace=True)
cols_to_impute = [c for c in evaluation_dataset.columns if c in pretrain.columns]
knn_impute_group(train_df=pretrain, test_df=evaluation_dataset, cols=cols_to_impute, n_neighbors=5)
display(evaluation_dataset)

K-Nearest Neighbors imputation applied...


,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,TRABSCOR,FAQ,mPACCdigit,mPACCtrailsB,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,ABETA,TAU,PTAU,FDG
0,90.6,0.8,17,0.6,7.0,29.330,41.330,8.0,20.0,8.0,0.0,2.0,100.00000,0.0,300.0,16.0,-19.747900,-18.115400,87118.40,5696.20,934054.4,2730.0,13399.4,14803.6,1612716.0,17.0,2.725,1.799998,1.580950,1.72,1.700000,1.80,1.942646,2.375,1.666670,1.500000,2.00,2.500000,2.50,2.052630,562.22,366.06,34.818,0.735309
1,66.3,0.4,16,0.0,0.5,3.670,7.670,4.0,28.0,46.0,6.0,4.0,33.33330,8.0,61.0,0.0,-4.268840,-2.718210,17827.40,7590.50,1000750.0,4368.0,16228.0,21865.0,1407420.0,28.0,2.000,1.222220,1.000000,1.40,1.166670,1.75,1.421050,1.000,1.000000,1.000000,1.00,1.000000,1.00,1.000000,1660.6,261.14,23.372,1.411922
2,64.5,0.4,18,0.0,0.5,3.330,5.330,2.0,29.0,53.0,2.0,3.0,25.00000,11.0,68.0,0.0,-0.420490,-0.088403,24943.58,7560.78,1010244.6,3963.4,17729.0,20967.8,1411200.0,26.4,1.375,2.111110,1.333330,1.60,1.400000,1.25,1.567570,1.250,1.111110,1.000000,1.00,1.166670,1.00,1.105260,1444.2,194.94,17.432,1.344564
3,60.3,0.8,16,1.0,2.0,8.330,15.330,7.0,28.0,35.0,3.0,7.0,87.50000,8.0,119.0,4.6,-6.496450,-5.972080,29464.50,7214.50,1153330.0,3661.0,20394.0,21953.0,1645580.0,24.0,3.250,2.222220,1.166670,2.00,3.333330,3.50,2.552630,2.825,1.577776,1.476192,2.00,2.263334,2.50,2.079702,499.74,353.16,36.6,1.336034
4,55.5,0.0,16,0.2,1.0,5.330,6.330,1.0,27.0,62.0,8.0,1.0,6.66667,15.0,71.0,5.0,-0.406210,-0.179726,18665.50,7222.50,954688.0,4131.0,15253.0,19195.0,1283140.0,26.0,2.625,2.333330,1.500000,1.80,1.833330,3.25,2.210530,1.875,1.333330,1.666670,1.40,1.500000,2.00,1.605260,1392.46,262.5,22.674,1.395548
5,68.9,0.0,15,0.8,0.0,1.670,5.670,4.0,30.0,46.0,4.0,8.0,72.72730,12.0,60.0,0.0,-0.341944,0.265948,41685.60,6796.20,996598.0,3984.0,15119.0,20880.0,1455600.0,24.0,1.375,1.333330,1.333330,1.40,1.000000,2.50,1.421050,1.750,1.222220,1.333330,1.00,1.000000,1.75,1.342110,1422.1,273.38,24.152,1.241280
6,67.7,0.6,14,2.0,2.5,18.600,27.000,7.4,25.0,29.8,2.8,5.8,83.49206,1.0,234.2,15.0,-14.443200,-14.443200,79008.30,6951.70,1137970.0,4197.0,22067.0,19711.0,1706490.0,18.6,2.450,2.066666,1.400000,1.56,1.783334,2.05,1.919884,3.375,3.111110,3.000000,4.00,3.666670,4.00,3.444440,634.0,461.94,51.582,0.948225
7,71.1,0.6,14,0.4,0.0,6.864,11.664,4.4,27.0,37.2,6.0,5.0,51.68182,8.0,122.2,0.2,-6.753610,-6.753610,29883.20,6816.70,999323.0,3874.0,16455.0,19162.0,1416240.0,21.4,2.350,1.666664,1.100000,1.16,1.266668,1.80,1.601154,1.925,1.511112,1.100000,1.15,1.280000,1.55,1.443120,848.66,188.6,17.23,1.303110
8,67.0,0.8,16,0.2,0.0,4.330,6.330,2.0,27.0,40.0,9.0,8.0,61.53850,10.0,75.0,0.2,-3.147900,-2.365620,60062.50,7596.60,1064780.0,4805.0,20048.0,20206.0,1573900.0,22.0,1.625,1.444440,1.000000,1.60,1.666670,2.00,1.526320,1.725,1.266666,1.000000,1.16,1.133334,1.45,1.297436,1522.4,205.88,18.88,1.242464
9,78.3,0.8,18,0.6,0.5,6.670,11.670,5.0,29.0,33.0,3.0,3.0,37.50000,3.0,116.0,0.0,-5.846740,-5.424100,27187.08,7415.76,1037789.0,3919.4,17844.2,19976.6,1527552.0,24.0,2.000,2.000000,1.166670,1.20,1.333330,1.75,1.631580,1.250,1.000000,1.000000,1.00,1.166670,1.50,1.131580,780.74,257.4,24.0,1.247442


In [6]:
preprocessor = ADNIPreprocessor()
evaluation_dataset = preprocessor.fit_transform(X=evaluation_dataset, y=y)
evaluation_dataset["DX"] = y
display(evaluation_dataset)

,AGE,PTGENDER,PTEDUCAT,APOE4,MMSE,CDRSB,ADAS13,LDELTOTAL,FAQ,MOCA,TRABSCOR,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,mPACCdigit,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,FDG,TAU/ABETA,PTAU/ABETA,Hippocampus/ICV,Entorhinal/ICV,Fusiform/ICV,MidTemp/ICV,Ventricles/ICV,WholeBrain/ICV,DX
0,91,1,17,1,20,7.0,41,0,16,17,300,8,0,100.00000,-19.747900,2.725,1.799998,1.580950,1.72,1.700000,1.80,2.375,1.666670,1.500000,2.00,2.500000,2.50,0.735309,0.651097,0.061929,0.003532,0.001693,0.008308,0.009180,0.054020,0.579181,NaN
1,66,0,16,0,28,0.5,8,8,0,28,61,46,6,33.33330,-4.268840,2.000,1.222220,1.000000,1.40,1.166670,1.75,1.000,1.000000,1.000000,1.00,1.000000,1.00,1.411922,0.157256,0.014074,0.005393,0.003104,0.011530,0.015536,0.012667,0.711053,NaN
2,64,0,18,0,29,0.5,5,11,0,26,68,53,2,25.00000,-0.420490,1.375,2.111110,1.333330,1.60,1.400000,1.25,1.250,1.111110,1.000000,1.00,1.166670,1.00,1.344564,0.134981,0.012070,0.005358,0.002808,0.012563,0.014858,0.017675,0.715877,NaN
3,60,1,16,1,28,2.0,15,8,5,24,119,35,3,87.50000,-6.496450,3.250,2.222220,1.166670,2.00,3.333330,3.50,2.825,1.577776,1.476192,2.00,2.263334,2.50,1.336034,0.706687,0.073238,0.004384,0.002225,0.012393,0.013341,0.017905,0.700865,NaN
4,56,0,16,0,27,1.0,6,15,5,26,71,62,8,6.66667,-0.406210,2.625,2.333330,1.500000,1.80,1.833330,3.25,1.875,1.333330,1.666670,1.40,1.500000,2.00,1.395548,0.188515,0.016283,0.005629,0.003219,0.011887,0.014959,0.014547,0.744025,NaN
5,69,0,15,1,30,0.0,6,12,0,24,60,46,4,72.72730,-0.341944,1.375,1.333330,1.333330,1.40,1.000000,2.50,1.750,1.222220,1.333330,1.00,1.000000,1.75,1.241280,0.192237,0.016983,0.004669,0.002737,0.010387,0.014345,0.028638,0.684665,NaN
6,68,1,14,2,25,2.5,27,1,15,19,234,30,3,83.49206,-14.443200,2.450,2.066666,1.400000,1.56,1.783334,2.05,3.375,3.111110,3.000000,4.00,3.666670,4.00,0.948225,0.728612,0.081360,0.004074,0.002459,0.012931,0.011551,0.046299,0.666848,NaN
7,71,1,14,0,27,0.0,12,8,0,21,122,37,6,51.68182,-6.753610,2.350,1.666664,1.100000,1.16,1.266668,1.80,1.925,1.511112,1.100000,1.15,1.280000,1.55,1.303110,0.222233,0.020303,0.004813,0.002735,0.011619,0.013530,0.021100,0.705617,NaN
8,67,1,16,0,27,0.0,6,10,0,22,75,40,9,61.53850,-3.147900,1.625,1.444440,1.000000,1.60,1.666670,2.00,1.725,1.266666,1.000000,1.16,1.133334,1.45,1.242464,0.135234,0.012401,0.004827,0.003053,0.012738,0.012838,0.038162,0.676523,NaN
9,78,1,18,1,29,0.5,12,3,0,24,116,33,3,37.50000,-5.846740,2.000,2.000000,1.166670,1.20,1.333330,1.75,1.250,1.000000,1.000000,1.00,1.166670,1.50,1.247442,0.329687,0.030740,0.004855,0.002566,0.011681,0.013078,0.017798,0.679380,NaN


## Load the Models

In [7]:
models_dir = "../results"
model_files = {
    "Model": "Model.pkl",
    "XAIModel": "XAIModel.pkl",
    "AltModel": "AltModel.pkl",
    "AltXAIModel": "AltXAIModel.pkl"
}

loaded_models = {}
for name, fname in model_files.items():
    path = os.path.join(models_dir, fname)
    mdl = joblib.load(path)
    if mdl is not None:
        loaded_models[name] = mdl


**Model** and **XAIModel** are built in the "*07_Classification_Models*" notebook. The alternative models, **AltModel** and **AltXAIModel**, are created in the "*Appendix_Alternative_Models*" notebook. These alternative versions were developed to evaluate model performance without the highly influential features `CDRSB`, `LDELTOTAL`, and `mPACCdigit`, which played a major role in the original models.


## Predict Diagnosis

In [8]:
X_eval_1 = evaluation_dataset.drop(['DX'], axis=1)
X_eval_2 = X_eval_1.drop(["CDRSB", "LDELTOTAL", "mPACCdigit"], axis=1)

# Prepare containers for predictions and probabilities (keep them OUTSIDE evaluation_dataset)
model_names = list(loaded_models.keys())
preds_df = pd.DataFrame(index=evaluation_dataset.index, columns=model_names)
probs_df = pd.DataFrame(index=evaluation_dataset.index, columns=model_names, dtype=float)

# Predict with each model, but DO NOT write Pred_* columns into evaluation_dataset ---
for name, mdl in loaded_models.items():
    # choose the correct input matrix for this model
    if name in ["Model", "XAIModel"]:
        X_eval = X_eval_1.copy()
    elif name in ["AltModel", "AltXAIModel"]:
        X_eval = X_eval_2.copy()
    else:
        X_eval = X_eval_1.copy()

    # Predict (DataFrame first, fallback to numpy array)
    try:
        preds = mdl.predict(X_eval)
    except Exception:
        preds = mdl.predict(X_eval.values)

    # Store predictions in external DataFrame (no changes to evaluation_dataset)
    preds_df[name] = pd.Series(preds, index=evaluation_dataset.index)

    # Try to get a top-probability per sample, store in probs_df (NaN if unavailable)
    top_probs = None
    if hasattr(mdl, "predict_proba"):
        try:
            probs = mdl.predict_proba(X_eval)
        except Exception:
            probs = mdl.predict_proba(X_eval.values)
        top_probs = np.max(probs, axis=1)
    elif hasattr(mdl, "decision_function"):
        try:
            dec = mdl.decision_function(X_eval)
        except Exception:
            dec = mdl.decision_function(X_eval.values)
        # convert decision scores to softmax as proxy
        try:
            exp = np.exp(dec - np.max(dec, axis=1, keepdims=True))
            softmax = exp / np.sum(exp, axis=1, keepdims=True)
            top_probs = np.max(softmax, axis=1)
        except Exception:
            top_probs = None

    if top_probs is not None:
        probs_df[name] = pd.Series(top_probs, index=evaluation_dataset.index)
    else:
        probs_df[name] = np.nan  # explicit float NaN

    # Print summary counts (optional)
    print(f"Model '{name}': predictions computed (not stored in evaluation_dataset).")
    try:
        vc = preds_df[name].value_counts(dropna=False)
        print(f" -> Prediction counts for {name}:\n{vc}\n")
    except Exception as e:
        print(f" -> Could not compute value_counts for {name}: {e}\n")

# Majority vote and apply to DX (no new Pred_* columns in evaluation_dataset) ---
vote_order = ["Model", "XAIModel", "AltModel", "AltXAIModel"]  # tie-breaker order

def majority_from_preds(row):
    # row is a Series containing predictions from each model (indexed by model name)
    preds = [row[m] for m in vote_order]
    # count frequencies, ignore NaN
    counts = pd.Series([p for p in preds if pd.notna(p)]).value_counts()
    if len(counts) == 0:
        return pd.NA  # no valid prediction
    if counts.iloc[0] > 1:
        return counts.idxmax()  # majority exists
    else:
        # no majority -> return the first model's prediction (if present), else first non-NaN
        first = preds[0]
        if pd.isna(first):
            # fallback to first non-NaN prediction
            for p in preds:
                if pd.notna(p):
                    return p
            return pd.NA
        return first

# Compute majority series from preds_df rows
majority_series = preds_df.apply(majority_from_preds, axis=1)

# Replace only existing NaNs in evaluation_dataset['DX'] (so we don't duplicate or erase non-NaN values)
if 'DX' in evaluation_dataset.columns:
    evaluation_dataset.loc[:, 'DX'] = evaluation_dataset['DX'].fillna(majority_series)
else:
    # If somehow DX did not exist, create it from majority
    evaluation_dataset.loc[:, 'DX'] = majority_series

# Print detailed per-index results (predictions from models and final DX) ---
for idx in evaluation_dataset.index:
    # gather preds for this index
    preds_for_idx = preds_df.loc[idx, vote_order].tolist()
    probs_for_idx = probs_df.loc[idx, vote_order].tolist()
    print(f"Index {idx}:")
    for m, p, pr in zip(vote_order, preds_for_idx, probs_for_idx):
        # show probability only if not NaN
        if pd.notna(pr):
            print(f"  {m}: {p}  (p={pr:.3f})")
        else:
            print(f"  {m}: {p}")
    print(f"  => Final DX: {evaluation_dataset.loc[idx, 'DX']}\n")

Model 'Model': predictions computed (not stored in evaluation_dataset).
 -> Prediction counts for Model:
Model
LMCI    3
EMCI    3
CN      3
AD      2
Name: count, dtype: int64

Model 'XAIModel': predictions computed (not stored in evaluation_dataset).
 -> Prediction counts for XAIModel:
XAIModel
LMCI    3
EMCI    3
CN      3
AD      2
Name: count, dtype: int64

Model 'AltModel': predictions computed (not stored in evaluation_dataset).
 -> Prediction counts for AltModel:
AltModel
CN      6
EMCI    3
AD      2
Name: count, dtype: int64

Model 'AltXAIModel': predictions computed (not stored in evaluation_dataset).
 -> Prediction counts for AltXAIModel:
AltXAIModel
CN      5
EMCI    4
AD      2
Name: count, dtype: int64

Index 0:
  Model: AD  (p=0.996)
  XAIModel: AD  (p=0.985)
  AltModel: AD  (p=0.341)
  AltXAIModel: AD  (p=0.910)
  => Final DX: AD

Index 1:
  Model: LMCI  (p=0.765)
  XAIModel: LMCI  (p=0.935)
  AltModel: CN  (p=0.301)
  AltXAIModel: CN  (p=0.705)
  => Final DX: LMCI

In

In [9]:
evaluation_dataset = evaluation_dataset.reindex(columns=['DX', "AGE", "PTGENDER", "PTEDUCAT", "APOE4", "MMSE", "CDRSB", "ADAS13", "LDELTOTAL", "FAQ", "MOCA", "TRABSCOR", "RAVLT_immediate", "RAVLT_learning", "RAVLT_perc_forgetting", "mPACCdigit", "EcogPtMem", "EcogPtLang", "EcogPtVisspat", "EcogPtPlan", "EcogPtOrgan", "EcogPtDivatt", "EcogSPMem", "EcogSPLang", "EcogSPVisspat", "EcogSPPlan", "EcogSPOrgan", "EcogSPDivatt", "FDG", "TAU/ABETA", "PTAU/ABETA", "Hippocampus/ICV", "Entorhinal/ICV", "Fusiform/ICV", "MidTemp/ICV", "Ventricles/ICV", "WholeBrain/ICV"])
display(evaluation_dataset)

,DX,AGE,PTGENDER,PTEDUCAT,APOE4,MMSE,CDRSB,ADAS13,LDELTOTAL,FAQ,MOCA,TRABSCOR,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,mPACCdigit,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,FDG,TAU/ABETA,PTAU/ABETA,Hippocampus/ICV,Entorhinal/ICV,Fusiform/ICV,MidTemp/ICV,Ventricles/ICV,WholeBrain/ICV
0,AD,91,1,17,1,20,7.0,41,0,16,17,300,8,0,100.00000,-19.747900,2.725,1.799998,1.580950,1.72,1.700000,1.80,2.375,1.666670,1.500000,2.00,2.500000,2.50,0.735309,0.651097,0.061929,0.003532,0.001693,0.008308,0.009180,0.054020,0.579181
1,LMCI,66,0,16,0,28,0.5,8,8,0,28,61,46,6,33.33330,-4.268840,2.000,1.222220,1.000000,1.40,1.166670,1.75,1.000,1.000000,1.000000,1.00,1.000000,1.00,1.411922,0.157256,0.014074,0.005393,0.003104,0.011530,0.015536,0.012667,0.711053
2,EMCI,64,0,18,0,29,0.5,5,11,0,26,68,53,2,25.00000,-0.420490,1.375,2.111110,1.333330,1.60,1.400000,1.25,1.250,1.111110,1.000000,1.00,1.166670,1.00,1.344564,0.134981,0.012070,0.005358,0.002808,0.012563,0.014858,0.017675,0.715877
3,LMCI,60,1,16,1,28,2.0,15,8,5,24,119,35,3,87.50000,-6.496450,3.250,2.222220,1.166670,2.00,3.333330,3.50,2.825,1.577776,1.476192,2.00,2.263334,2.50,1.336034,0.706687,0.073238,0.004384,0.002225,0.012393,0.013341,0.017905,0.700865
4,EMCI,56,0,16,0,27,1.0,6,15,5,26,71,62,8,6.66667,-0.406210,2.625,2.333330,1.500000,1.80,1.833330,3.25,1.875,1.333330,1.666670,1.40,1.500000,2.00,1.395548,0.188515,0.016283,0.005629,0.003219,0.011887,0.014959,0.014547,0.744025
5,CN,69,0,15,1,30,0.0,6,12,0,24,60,46,4,72.72730,-0.341944,1.375,1.333330,1.333330,1.40,1.000000,2.50,1.750,1.222220,1.333330,1.00,1.000000,1.75,1.241280,0.192237,0.016983,0.004669,0.002737,0.010387,0.014345,0.028638,0.684665
6,AD,68,1,14,2,25,2.5,27,1,15,19,234,30,3,83.49206,-14.443200,2.450,2.066666,1.400000,1.56,1.783334,2.05,3.375,3.111110,3.000000,4.00,3.666670,4.00,0.948225,0.728612,0.081360,0.004074,0.002459,0.012931,0.011551,0.046299,0.666848
7,CN,71,1,14,0,27,0.0,12,8,0,21,122,37,6,51.68182,-6.753610,2.350,1.666664,1.100000,1.16,1.266668,1.80,1.925,1.511112,1.100000,1.15,1.280000,1.55,1.303110,0.222233,0.020303,0.004813,0.002735,0.011619,0.013530,0.021100,0.705617
8,CN,67,1,16,0,27,0.0,6,10,0,22,75,40,9,61.53850,-3.147900,1.625,1.444440,1.000000,1.60,1.666670,2.00,1.725,1.266666,1.000000,1.16,1.133334,1.45,1.242464,0.135234,0.012401,0.004827,0.003053,0.012738,0.012838,0.038162,0.676523
9,LMCI,78,1,18,1,29,0.5,12,3,0,24,116,33,3,37.50000,-5.846740,2.000,2.000000,1.166670,1.20,1.333330,1.75,1.250,1.000000,1.000000,1.00,1.166670,1.50,1.247442,0.329687,0.030740,0.004855,0.002566,0.011681,0.013078,0.017798,0.679380


## Creation of NEWADNIMERGE.csv to accumulate data

We then create a dataset called **NEWADNIMERGE.csv**, which contains all the data from *train* and *test*. This dataset will then store the values ​​from the main app, ***main.py***.

In [10]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
new_dataset = pd.concat([train, test], ignore_index=True)

# Saving to new CSV
new_dataset.to_csv("../data/NEWADNIMERGE.csv", index=False)

print("File saved as NEWADNIMERGE.csv")

File saved as NEWADNIMERGE.csv


In [11]:
display(new_dataset)

,DX,AGE,PTGENDER,PTEDUCAT,APOE4,MMSE,CDRSB,ADAS13,LDELTOTAL,FAQ,MOCA,TRABSCOR,RAVLT_immediate,RAVLT_learning,RAVLT_perc_forgetting,mPACCdigit,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,FDG,TAU/ABETA,PTAU/ABETA,Hippocampus/ICV,Entorhinal/ICV,Fusiform/ICV,MidTemp/ICV,Ventricles/ICV,WholeBrain/ICV
0,AD,80,1,14,0,21,6.5,42,0,19,13,300,15,1,100.0000,-20.06920,3.142860,3.000000,3.000000,3.200000,2.500000,2.75,4.000000,3.444440,2.666670,3.000000,3.666670,3.750000,1.042620,0.630939,0.061115,0.003517,0.000970,0.009514,0.011188,0.039862,0.680575
1,LMCI,82,1,20,0,24,1.5,20,2,4,20,155,29,0,83.3333,-10.20060,2.100000,1.780554,1.274284,1.373334,1.699998,1.65,2.950000,2.530556,2.014286,1.756666,2.316666,2.750000,1.080580,0.421770,0.041719,0.003360,0.001276,0.008863,0.010643,0.049299,0.640478
2,LMCI,71,1,19,0,26,1.0,8,2,2,25,106,51,2,-18.1818,-5.90200,2.750000,2.555560,2.285710,3.200000,3.833330,3.50,1.510714,1.266666,1.438096,1.760000,1.680000,1.850000,1.414550,0.182691,0.016757,0.005090,0.002786,0.012161,0.014206,0.025251,0.630110
3,CN,75,0,20,0,30,0.0,6,19,0,26,58,61,7,20.0000,3.19941,1.750000,1.333330,1.000000,1.000000,1.166670,1.00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.118820,0.263255,0.024724,0.004908,0.002758,0.012983,0.013293,0.044823,0.677111
4,CN,81,0,19,0,29,0.0,8,11,0,29,54,54,7,28.5714,-1.16303,1.875000,1.555560,1.000000,1.000000,1.333330,1.75,1.375000,1.111110,1.000000,1.200000,1.166670,1.750000,1.270141,0.197315,0.018091,0.004700,0.002912,0.013675,0.016983,0.036393,0.726675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,EMCI,87,0,12,0,25,0.5,10,9,0,27,124,44,9,30.7692,-7.36038,1.375000,1.444440,1.000000,1.200000,1.666670,1.25,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.210240,0.465307,0.043280,0.005005,0.002397,0.012850,0.014694,0.014110,0.724017
2414,AD,80,0,18,1,26,4.5,25,7,3,19,300,32,0,66.6667,-9.80261,2.610714,2.194446,1.671428,1.280000,1.800000,1.95,3.200000,2.647222,1.851430,2.640000,2.633334,2.633332,1.206340,0.819098,0.086348,0.003946,0.001983,0.009784,0.010737,0.018140,0.630939
2415,CN,84,1,20,1,30,0.0,9,13,0,25,79,59,8,14.2857,2.28550,1.500000,1.555560,1.000000,1.000000,1.666670,1.75,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.238840,0.405536,0.040640,0.004461,0.002830,0.011327,0.012686,0.029255,0.656308
2416,LMCI,56,1,16,0,29,2.0,8,4,1,26,108,31,4,37.5000,-2.84853,2.125000,1.622220,1.333334,1.600000,1.500000,1.90,1.850000,1.355554,1.100000,1.440000,1.400000,1.900000,1.347934,0.111505,0.009532,0.005026,0.002461,0.010543,0.013734,0.014713,0.726884
